In [45]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm.auto import tqdm  # progress bar for notebooks
from hardware_accelerators.nn import load_model

# from pytorch2tikz import Architecture
import onnx

In [3]:
# Device configuration with MPS support on macOS
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Hyperparameters
input_size = 28 * 28  # MNIST images are 28x28
hidden_size = 128
num_classes = 10
num_epochs = 5
batch_size = 64
learning_rate = 0.001

In [4]:
# Define the MLP model with one hidden layer using ReLU
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        out = self.fc2(x)
        return out

In [23]:
# Set random seeds for reproducibility
torch.manual_seed(42)

# Device configuration
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"{device=}")

# Hyperparameters
input_size = 28 * 28  # MNIST images are 28x28
hidden_size = 128
num_classes = 10
num_epochs = 5
batch_size = 32
learning_rate = 0.001

device=device(type='mps')


In [24]:
# Training function for one epoch
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}", leave=False)
    for batch_idx, (data, target) in enumerate(progress_bar):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch} Train Loss: {avg_loss:.4f}")

In [25]:
# Evaluation function
def evaluate(model, device, data_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            # Get predictions from the maximum value
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    avg_loss = running_loss / len(data_loader)
    accuracy = 100.0 * correct / total
    return avg_loss, accuracy

In [8]:
model = MLP(input_size, hidden_size, num_classes).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
fc1.weight 	 torch.Size([128, 784])
fc1.bias 	 torch.Size([128])
fc2.weight 	 torch.Size([10, 128])
fc2.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': None, 'params': [0, 1, 2, 3]}]


In [ ]:
# Data transformation: convert images to tensor and normalize them
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

# Load the MNIST training dataset (we will later perform CV on this)
train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)

# Train the model on the full training dataset
print("Training on the full training set...")
full_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(1, num_epochs + 1):
    train(model, device, full_train_loader, optimizer, criterion, epoch)

# Evaluate on the test set
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
test_loss, test_accuracy = evaluate(model, device, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%\n")

Training on the full training set...


NameError: name 'optimizer' is not defined

In [ ]:
# Data transformation: convert images to tensor and normalize them
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        # transforms.Normalize((0.1307,), (0.3081,))
    ]
)


train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
loader = DataLoader(train_dataset, batch_size=1)

data, target = iter(loader).__next__()

In [63]:
data

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [6]:
model_dir = "models"
pytorch_output = os.path.join(model_dir, "mlp_mnist.pth")
onnx_output = os.path.join(model_dir, "mlp_mnist.onnx")

model.eval()

# Save pytorch model
torch.save(model.state_dict(), pytorch_output)
print(f"Model saved to 'mlp_mnist.pth'.\n")

# Export the final model to ONNX format
dummy_input = torch.randn(1, 1, 28, 28, device=device)
torch.onnx.export(
    model,
    dummy_input,
    onnx_output,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11,
)
print(f"Model exported to ONNX format at '{onnx_output}'.\n")

# Loading the trained model


In [ ]:
from hardware_accelerators.nn import load_model

model = load_model("models/mlp_mnist.pth")

In [52]:
model.graph.output

[name: "output"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "batch_size"
      }
      dim {
        dim_value: 10
      }
    }
  }
}
]

In [15]:
# Print basic model info
print(f"Model IR version: {model.ir_version}")
print(f"Opset version: {model.opset_import[0].version}")
print(f"Producer name: {model.producer_name}")

# Get graph
graph = model.graph

# Print input info
print("\nInputs:")
for input in graph.input:
    print(f"- {input.name}: {input.type.tensor_type.elem_type}")
    shape = [dim.dim_value for dim in input.type.tensor_type.shape.dim]
    print(f"  Shape: {shape}")

# Print output info
print("\nOutputs:")
for output in graph.output:
    print(f"- {output.name}: {output.type.tensor_type.elem_type}")
    shape = [dim.dim_value for dim in output.type.tensor_type.shape.dim]
    print(f"  Shape: {shape}")

Model IR version: 6
Opset version: 11
Producer name: pytorch

Inputs:
- input: 1
  Shape: [0, 1, 28, 28]

Outputs:
- output: 1
  Shape: [0, 10]


In [16]:
# Print operators used in the model
print("Operators used in the model:")
ops = {node.op_type for node in graph.node}
for op in sorted(ops):
    print(f"- {op}")

# Print initializers (weights and biases)
print("\nInitializers (weights and biases):")
for init in graph.initializer:
    # Convert to numpy array to get shape and data type
    np_array = onnx.numpy_helper.to_array(init)
    print(f"- {init.name}:")
    print(f"  Shape: {np_array.shape}")
    print(f"  Data type: {np_array.dtype}")

Operators used in the model:
- Flatten
- Gemm
- Relu

Initializers (weights and biases):
- fc1.weight:
  Shape: (128, 784)
  Data type: float32
- fc1.bias:
  Shape: (128,)
  Data type: float32
- fc2.weight:
  Shape: (10, 128)
  Data type: float32
- fc2.bias:
  Shape: (10,)
  Data type: float32
